## 웹스크래핑을 이용한 데이터 수집
* open API를 이용한 데이터 수집이 불가한 경우에 해당
* 방식 : 웹을 요청하여 html을 모두 받아서 DOM으로 올려서 데이터를 추출방식
* DOM을 띄어서 데이터를 추출할 때 사용하는 라이브러리
 > beautifulsoup (bs4)를 활용
 > conda install beautifulsoup4 (오픈터미널로 설치하면 된다)

In [101]:
from bs4 import BeautifulSoup

In [102]:
from urllib.request import urlopen

In [103]:
# 네이버 금융 -> 환율 고시 더보기 -> 통화명 컬럼에서 오른마우스에서 프레임 소스 보기 클릭 -> 주소 확인

In [104]:
# 네이버 영화 -> 영화랭킹 -> 

- 콘텐츠가 존재하는 해당 페이지까지 진입
- 진입간에 로그인, ajax등등 사람의 손을 타지 않는지 체크
- 그냥 url만 넣으면 화면이 구성된다 => ok
- html 자체에 프레임이 적용된 경우 실제 주소까지 찾아서 이동
- 통신시 get, post등 데이터를 전달해서 획득하는 것도 ok

### 네이버 금융 > 고시환율정보 수집

- 하루에 환율이 수시로 변경된다 
- 그래서, 그 주기를 관찰하여서 수집 주기를 결정해야 한다 

In [105]:
target_site = 'https://finance.naver.com/marketindex/exchangeList.nhn'
target_site

'https://finance.naver.com/marketindex/exchangeList.nhn'

In [106]:
# 요청 및 응답 획득
page = urlopen( target_site )
page
# 응답이 온다

In [107]:
# DOM 구성  ( 스크랩핑을 하는 과정이다 -> 스크랩핑이 성공된다 )
# 'html5lib' 파서는 html 양이 크거나, 정교한 파싱을 할때 즉, 아래 파서가
# 정상적으로 결과를 내지 못하면 이 파서로 교체
soup = BeautifulSoup( page,'html.parser' ) 

In [108]:
# find: 한놈 찾기, find_all : 다 찾기
for td in soup.find_all('tb', 'tit'):
    print( td.a.string.strip() )

- 데이터를 추출하여 디비에 적제하기 위해 최종 형태는 다음과 같다
- [
    {
        'name':'미국 USD',
        'code':'USD',
        'buy_std_rate':1209.90,
        'cash_sell':1188.73
    },{}
  ]  

In [109]:
# 통화
# 리스트 내포를 통해서 간단하게 가공해 보았다
tmp = [ td.a.string.strip() for td in soup.find_all('td', 'tit')]
tmp[:3]

['미국 USD', '유럽연합 EUR', '일본 JPY (100엔)']

In [110]:
# 매매 기준율(가지고 올 때 웹에서의 소스검사에서 부모코드를 보면된다)
tmp = [ td.string.strip() for td in soup.find_all('td', 'sale')]
tmp[:3]

['1,209.30', '1,353.45', '1,141.55']

In [111]:
# body > div > tbody > tr:nth-child(1) > tr:nth-child(4)

- 만약 대상 페이지에 table이 n개 존재하면 특정해서 찾아야한다
- 대상 데이터가 있는 tr을 모두 찾았다
- table.tbl_exchange > tbody tr
- 위의 표현을 for문으로 구동하고 하나하나의 tr에서 tr:nth-child(4)(서열4위)를 선택 

In [112]:
# css selector -> select: 전부 뽑기(리스트시킴), select_one(리스트 없음) : 처음 만나는 한개
# 원하는 데이터를 감싸는 tr을 찾아서 그 밑에서 자식들을 탐색후 거기서 데이터를 추출!!
for tr in soup.select('table.tbl_exchange > tbody > tr'):
    # 데이터 한줄한줄 뽑아서 => tr에서 탐색하여 세부 데이터 추출 -> for문으로 하위 코드들도 불러온다( tr.select : tag 문법(tag.name))
    print( tr.select_one('td.tit').a.string.strip() ) 
    print( tr.select_one('td.sale').string.strip() )
    print( tr.select_one('td:nth-child(4)').string.strip() )

미국 USD
1,209.30
1,188.14
유럽연합 EUR
1,353.45
1,326.52
일본 JPY (100엔)
1,141.55
1,121.58
중국 CNY
170.82
162.28
홍콩 HKD
154.21
151.18
대만 TWD
38.72
36.01
영국 GBP
1,468.45
1,439.53
오만 OMR
3,140.96
2,952.51
캐나다 CAD
914.27
896.26
스위스 CHF
1,241.58
1,217.13
스웨덴 SEK
126.08
123.00
호주 AUD
823.96
807.73
뉴질랜드 NZD
784.23
768.79
체코 CZK
52.48
47.76
칠레 CLP
1.70
1.57
터키 TRY
220.62
0.00
몽골 MNT
0.45
0.00
이스라엘 ILS
347.64
319.83
덴마크 DKK
181.39
176.95
노르웨이 NOK
135.60
132.28
사우디아라비아 SAR
322.34
300.10
쿠웨이트 KWD
3,976.00
3,657.92
바레인 BHD
3,207.69
2,951.08
아랍에미리트 AED
329.23
306.52
요르단 JOD
1,705.64
1,569.19
이집트 EGP
73.00
0.00
태국 THB
39.35
36.99
싱가포르 SGD
875.04
857.63
말레이시아 MYR
288.93
267.55
인도네시아 IDR 100
8.53
7.68
카타르 QAR
332.13
0.00
카자흐스탄 KZT
3.12
0.00
브루나이 BND
875.04
822.54
인도 INR
17.16
0.00
파키스탄 PKR
7.56
0.00
방글라데시 BDT
14.33
0.00
필리핀 PHP
23.30
21.39
멕시코 MXN
62.26
57.04
브라질 BRL
308.57
283.89
베트남 VND 100
5.21
4.60
남아프리카 공화국 ZAR
80.40
73.97
러시아 RUB
18.57
16.53
헝가리 HUF
4.17
3.84
폴란드 PLN
313.19
288.14


In [113]:
results = []
for tr in soup.select('table.tbl_exchange > tbody > tr'):
    tmp = dict()
    tmp['name']         = tr.select_one('td.tit').a.string.strip()  
    tmp['code']         = tr.select_one('td.tit').a['href'][-6:-3]
    tmp['buy_std_rate'] = tr.select_one('td.sale').string.strip() 
    tmp['cash_sell']    = tr.select_one('td:nth-child(4)').string.strip() 
    # :thh-child(4)가 안되면
    # tmp['cash_sell']  = tr.select_one('td')[3].string.strip() 
    results.append( tmp )
results

[{'name': '미국 USD',
  'code': 'USD',
  'buy_std_rate': '1,209.30',
  'cash_sell': '1,188.14'},
 {'name': '유럽연합 EUR',
  'code': 'EUR',
  'buy_std_rate': '1,353.45',
  'cash_sell': '1,326.52'},
 {'name': '일본 JPY (100엔)',
  'code': 'JPY',
  'buy_std_rate': '1,141.55',
  'cash_sell': '1,121.58'},
 {'name': '중국 CNY',
  'code': 'CNY',
  'buy_std_rate': '170.82',
  'cash_sell': '162.28'},
 {'name': '홍콩 HKD',
  'code': 'HKD',
  'buy_std_rate': '154.21',
  'cash_sell': '151.18'},
 {'name': '대만 TWD',
  'code': 'TWD',
  'buy_std_rate': '38.72',
  'cash_sell': '36.01'},
 {'name': '영국 GBP',
  'code': 'GBP',
  'buy_std_rate': '1,468.45',
  'cash_sell': '1,439.53'},
 {'name': '오만 OMR',
  'code': 'OMR',
  'buy_std_rate': '3,140.96',
  'cash_sell': '2,952.51'},
 {'name': '캐나다 CAD',
  'code': 'CAD',
  'buy_std_rate': '914.27',
  'cash_sell': '896.26'},
 {'name': '스위스 CHF',
  'code': 'CHF',
  'buy_std_rate': '1,241.58',
  'cash_sell': '1,217.13'},
 {'name': '스웨덴 SEK',
  'code': 'SEK',
  'buy_std_rate': '

In [115]:
results = [ {
    'name':tr.select_one('td.tit').a.string.strip(),
    'code':tr.select_one('td.tit').a['href'][-6:-3],
    'buy_std_rate':tr.select_one('td.sale').string.strip(),
    'cash_sell':tr.select_one('td:nth-child(4)').string.strip()
} for tr in soup.select('table.tbl_exchange > tbody > tr') 
  if tr.select_one('td.tit').a.string.strip().count('JYP') == 0
]


In [116]:
results

[{'name': '미국 USD',
  'code': 'USD',
  'buy_std_rate': '1,209.30',
  'cash_sell': '1,188.14'},
 {'name': '유럽연합 EUR',
  'code': 'EUR',
  'buy_std_rate': '1,353.45',
  'cash_sell': '1,326.52'},
 {'name': '일본 JPY (100엔)',
  'code': 'JPY',
  'buy_std_rate': '1,141.55',
  'cash_sell': '1,121.58'},
 {'name': '중국 CNY',
  'code': 'CNY',
  'buy_std_rate': '170.82',
  'cash_sell': '162.28'},
 {'name': '홍콩 HKD',
  'code': 'HKD',
  'buy_std_rate': '154.21',
  'cash_sell': '151.18'},
 {'name': '대만 TWD',
  'code': 'TWD',
  'buy_std_rate': '38.72',
  'cash_sell': '36.01'},
 {'name': '영국 GBP',
  'code': 'GBP',
  'buy_std_rate': '1,468.45',
  'cash_sell': '1,439.53'},
 {'name': '오만 OMR',
  'code': 'OMR',
  'buy_std_rate': '3,140.96',
  'cash_sell': '2,952.51'},
 {'name': '캐나다 CAD',
  'code': 'CAD',
  'buy_std_rate': '914.27',
  'cash_sell': '896.26'},
 {'name': '스위스 CHF',
  'code': 'CHF',
  'buy_std_rate': '1,241.58',
  'cash_sell': '1,217.13'},
 {'name': '스웨덴 SEK',
  'code': 'SEK',
  'buy_std_rate': '